---
---

# <center><font color='black'>Rejection ABC</font></center> <a class="tocSkip">
    
### <center><font color='black'>Nicolai Haug</font></center> <a class="tocSkip">
    
### <center><font color='black'>2021</font></center> <a class="tocSkip">
    
---
---

# Table of Contents <a class="tocSkip">

* [Introduction](#introduction)
    * [Configure Notebook](#configure) 
* [Rejection ABC](#rejection)

# Introduction <a name="introduction"></a>


Approximate Bayesian Computation (ABC) constitutes a class of computational methods rooted in Bayesian statistics that can be used to evaluate posterior distributions of model parameters without having to explicitly calculate likelihoods. ABC methods approximate the likelihood function by assessing how likely it is the model could have produced the observed data, based on comparing synthetic data generated by the simulator to the observed data. The simulations that do not reproduce the observed data within a specified tolerance are discarded [[1]](#references) [[2, p. ix]](#references).

ABC methods have been successfully applied to a wide range of real-world problems, and have also paved the way for a range of other likelihood-free approaches. However, even though ABC methods are mathematically well-founded, they inevitably make assumptions and approximations whose impact needs to be carefully assessed [[1]](#references). In the following, we take a look at the vanilla rejection ABC algorithm and its history.  

## Configure Notebook <a name="configure"></a>

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-exclamation-circle" aria-hidden="true"></i>&nbsp; Important</b><br>
    <p style="color: black">
        Run the cell below to configure the notebook. 
    </p>
<div>
</div>
</div>

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns

from IPython.display import display
from latex_envs.latex_envs import figcaption 
from matplotlib import cm
from matplotlib import gridspec
from matplotlib.ticker import FormatStrFormatter, LinearLocator
from mpl_toolkits.mplot3d import Axes3D
from numpy.random import default_rng

import warnings
# Comment this to turn on warnings
warnings.filterwarnings('ignore')

#plt.style.use('seaborn')
sns.set()
sns.set_context("paper")
sns.set_style("darkgrid", {"axes.facecolor": "0.96"})

# Set fontsizes in figures
params = {'legend.fontsize': 'large',
          'axes.labelsize': 'large',
          'axes.titlesize': 'large',
          'xtick.labelsize': 'large',
          'ytick.labelsize': 'large',
          'legend.fontsize': 'large',
          'legend.handlelength': 2}
plt.rcParams.update(params)
plt.rc('text', usetex=True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

# Rejection ABC <a name="rejection"></a>





%===============================================================
\subsubsection{Rejection ABC}
%===============================================================

Given observed data $y_0$ and synthetic data $y$ generated by a simulator, let $\rho (\cdot, \cdot)$ be a distance metric (e.g., the Euclidean norm) defined in data space $\mathcal{R}^D$ and $\epsilon \geq 0$ be a tolerance. For small $\epsilon$, the ABC approximation to the posterior is

\begin{equation}
    p \left(\theta \mid \y = y_0 \right) \simeq p \left(\theta \mid \rho \left(y, y_0 ) \leq \epsilon ) 
\end{equation}

Rejection ABC is a rejection-sampling method for obtaining independent samples from the approximate posterior $p \qty(\bm{\theta} \mid \rho \qty(\bm{x}, \bm{x}_0 ) \leq \epsilon )$. It works by first sampling a set of parameters from the prior $p(\bm{\theta})$, then simulating data under the model specified by the sampled parameters, and only accepting and retaining the sample if the distance between $\bm{x}$ and $\bm{x}_0$ is no more than $\epsilon$. The tolerance parameter $\epsilon$ controls the trade-off between estimation accuracy and computational efficiency. With sufficiently small $\epsilon$, and a sensible distance metric, the accepted samples follow the exact posterior more closely, though the algorithm accepts less often. On the other hand, the algorithm accepts more often with a large $\epsilon$, but the accepted samples will yield a replica of the prior \cite[p. 58]{papamakarios2019neural} \cite{abc_handbook}. 

An issue with ABC in general is that the required number of simulations increases dramatically as $\epsilon$ becomes small. Moreover, likelihood-free inference also becomes challenging when the dimensionality of the data is large. A common approach to lessen this problem is to use lower-dimensional summary statistics, $S(\bm{x})$ and $S(\bm{x}_0)$, that capture important features such as the mean and standard deviation, in place of raw data \cite{SNL18}. A further motivation for this approach is that real-world experiments often are interested in capturing summary statistics of the experimental data. A summary statistic that contains the same amount of information about model parameters as the whole dataset, is referred to as being a \textit{sufficient statistic} \cite{ABCprimer}. The acceptance criterion in the rejection ABC algorithm then becomes:

\begin{equation}
    \rho \qty(S(\bm{x}), S(\bm{x}_0))
\end{equation}

**Algorithms** 

[ABC handbook]

For discrete data $\mathcal{D}$, probability model $\mathcal{M}$ with parameters $\theta$ having prior $\pi(\theta)$, we can simulate observations from: 

\begin{equation}
    \pi (\theta \mid \mathcal{D}) \propto p(\mathcal{D} \mid \theta) \pi(\theta),
\end{equation}

via:

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-calculator" aria-hidden="true"></i>&nbsp; Algorithm A</b><br>
<div>
    
* __A1:__ Generate $\theta \sim \pi(\theta)$
* __A2:__ Accept $\theta$ with probability proportional to the likelihood $p(\mathcal{D}\mid \theta)$

</div>
</div>

Algorithm A can be extended dramatically in its usefulness using the following, stochastically equivalent, version:

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-calculator" aria-hidden="true"></i>&nbsp; Algorithm B (Rubin, 1984)</b><br>
<div>
    
* __B1:__ Generate $\theta \sim \pi(\theta)$
* __B2:__ Simulate an observation $\mathcal{D}'$ from model $\mathcal{M}$ with parameter $\theta$
* __B3:__ Accept $\theta$ if $\mathcal{D}' = \mathcal{D}$

</div>
</div>

While algorithms A and B are probabilistically identical, B is much more general in that one does not need to compute probabilities explicitly to make it work; only simulation is needed. Version B is due to Rubin (1984). 

The drawback of B is clear. It will typically be the case that for a given value of $\theta$, the chance of the outcome $\mathcal{D}'=\mathcal{D}$, namely $p(\mathcal{D} \mid \theta)$, is either vanishingly small or very time consuming to compute, resulting in an algorithm that does not work effectively. This is where ABC finally comes into play, in the form of the following scheme. We start with a discrepancy metric $\rho$ to compare datasets and a tolerance $\epsilon \geq 0$, and then: 

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-calculator" aria-hidden="true"></i>&nbsp; Algorithm C (Pritchard et al., 1999)</b><br>
<div>
    
* __C1:__ Generate $\theta \sim \pi(\theta)$
* __C2:__ Simulate an observation $\mathcal{D}'$ from model $\mathcal{M}$ with parameter $\theta$
* __C3:__ Compute $\rho \equiv \rho (\mathcal{D}', \mathcal{D})$, and accept $\theta$ as an appropriate draw from $\pi (\theta \mid \mathcal{D})$ if $\rho \leq \epsilon$

</div>
</div>

The parameter $\epsilon$ measures the tension between computability and accuracy. If $\rho$ is a metric, then 

\begin{equation*}
    \rho = 0 \quad \implies \quad \mathcal{D}'=\mathcal{D},
\end{equation*}

so that such an accepted $\theta$ is indeed an observation from the true posterior. 

Pritchard et al. (1999) were the first to describe a version of this scheme, in which the datasets in C3 were compared through a choice of summary statistics. Thus, $\rho$ compares how well a set of simulated summary statistics matches the observed summary statistics. If the statistics are sufficient for $\theta$, then when $\epsilon=0$, the accepted values of $\theta$ are still from the true posterior based on the full data. 

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-bug" aria-hidden="true"></i>&nbsp; Inquiry</b><br>
    <p style="color: black">
        This begs the question of how one might identify 'approximately sufficient' statistics, a topic covered in <a href="https://github.com/nicolossus/Master-thesis/blob/master/notebooks/development/K.%20Summary%20Statistics.ipynb">this notebook</a>.        
    </p>
</div>

# Rejection ABC Code

In [2]:
def rejection_ABC(observed_data, priors, n_posterior_samples, n_sims_per_parameter, epsilon): 
    """ 
    Rejection ABC as described by the Pritchard et al. (1999) algorithm. 
    
    This implementation expects predefined function objects: 
        1. simulator(*thetas, n_sims_per_parameter)
        2. summary_calculator(data)
        3. distance_metric(sim_sumstat, obs_sumstat)
    
    Arguments
    ---------
    observed_data : array
        The observed data
    priors : list
        List of priors as scipy.stats objects
    n_posterior_samples : int
        The number of posterior samples to produce
    n_sims_per_parameter : int
        Image filename (with path included) of image with shape (H, W, c) to
        transform
    epsilon : float
        Discrepancy threshold
          
    Returns
    -------
    samples : list 
        ABC posterior samples
    """
    
    # calculate observed data summary statistic
    obs_sumstat = summary_calculator(observed_data)
    
    samples = []
    accepted_count = 0 
    
    while accepted_count < n_posterior_samples: 
        # draw thetas from priors
        thetas = [theta.rvs() for theta in priors]
        # simulated data given realizations of drawn thetas
        sim_data = simulator(*thetas, N)
        # summary stat of simulated data
        sim_sumstat = summary_calculator(sim_data)
        # calculate discrepancy
        distance = distance_metric(sim_sumstat, obs_sumstat)
        # rejection step
        if distance <= epsilon:
            accepted_count += 1
            samples.append(thetas)
    
    return samples

Most well-tested implementations will do a bit more than this under the hood, but the preceding function gives the gist of the expectation–maximization approach.

# Goodness of Fit

Assessing performance 

Expected quadratic loss

https://en.wikipedia.org/wiki/Loss_function

Expected loss

In some contexts, the value of the loss function itself is a random quantity because it depends on the outcome of a random variable X.

Both frequentist and Bayesian statistical theory involve making a decision based on the expected value of the loss function; however, this quantity is defined differently under the two paradigms.

# TODOs 

- Sufficient vs insufficient statistics 
- $p(\theta \mid \rho (\hat{D}, D) \leq \epsilon)$ and $p(\theta |D)$ as a function of $\epsilon$. 
- The accuracy of the posterior (defined as the expected quadratic loss) delivered by ABC as a function of $\epsilon$
- Accuracy as a function of the number of data points in observed data
- KL divergence
- Accuracy vs number of posterior samples

# References <a name="references"></a>

[1] Mikael Sunnåker et al. “Approximate Bayesian Computation”. eng. In: 9.1 (2013), e1002803. issn: 1553-734X 

[2] “Handbook of Approximate Bayesian Computation”. eng. 1st ed. CRC Press, 2018. isbn: 9781439881507 